# Kayıp Fonksiyonları

Bu alıştırmada, Kayıp fonksiyonlarının `LinearRegression` modeli üzerindeki etkilerini karşılaştıracaksınız.

👇 Bu zorluk için kullanmak üzere bir CSV dosyası indirelim ve onu bir DataFrame'e dönüştürelim

In [1]:
import pandas as pd

data = pd.read_csv("https://d32aokrjazspmn.cloudfront.net/materials/loss_functions_dataset.csv")
data.sample(5)

,Relative Compactness,Surface Area,Wall Area,Roof Area,Overall Height,Glazing Area,Average Temperature
178,0.69,735.0,294.0,220.5,3.5,0.10,12.885
456,0.74,686.0,245.0,220.5,3.5,0.25,13.620
78,0.71,710.5,269.5,220.5,3.5,0.10,12.285
360,0.74,686.0,245.0,220.5,3.5,0.25,13.540
356,0.76,661.5,416.5,122.5,7.0,0.25,36.910


🎯 Göreviniz, tasarımına göre bir seranın içindeki ortalama sıcaklığı tahmin etmektir. Sıcaklık tahminleriniz, her bir bitki için iklim ihtiyaçlarına göre uygun sera tasarımını seçmenize yardımcı olacaktır.

🌿 Bitkilerin küçük sıcaklık değişimlerini kaldırabildiğini, ancak sıcaklık değişimleri arttıkça katlanarak daha duyarlı hale geldiğini biliyorsunuz.

## 1. Teori

❓ Teorik olarak, bitkileri öldürme riskini sınırlamak için modelinizi hangi Kayıp fonksiyonu üzerinde eğitirsiniz?

<details>
<summary> 🆘 Cevap </summary>
    
Teorik olarak, Ortalama Kare Hata (MSE) Kayıp fonksiyonunu kullanırsınız. Bu, aykırı tahminleri cezalandırır ve modelinizin büyük hatalar yapmasını engeller. Bu, daha küçük sıcaklık değişimleri ve bitkiler için daha düşük risk sağlayacaktır.

</details>

> Bitkiyi öldüren “büyük sıcaklık hataları” küçük hatalara göre çok daha önemli olduğu için, quadratic penalize eden MSE kayıp fonksiyonu kullanılır

## 2. Uygulama

### 2.1 Ön İşleme

❓ Özellikleri standartlaştırın

In [2]:
from sklearn.preprocessing import StandardScaler
X = data.loc[:,'Relative Compactness':'Glazing Area']
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

### 2.2 Modelleme

Bu bölümde, farklı Kayıp fonksiyonları üzerinde optimize edilmiş modelleri değerlendirerek teoriyi doğrulayacaksınız.

### En Küçük Kareler (MSE) Kaybı

❓ **En Küçük Kareler Kaybı** (MSE) üzerinde **Stokastik Gradyan İnişi** (SGD) ile optimize edilmiş bir Doğrusal Regresyon modelini **10-Katlı Çapraz doğrula**

In [ ]:
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.linear_model import SGDRegressor
sgd_model = SGDRegressor(loss="squared_error")
sgd_model_cv = cross_validate(sgd_model,  X_scaled, data['Average Temperature'],cv = 10, scoring = ['r2','max_error'])
sgd_model_cv

{'fit_time': array([0.00375986, 0.00171113, 0.0016861 , 0.00155616, 0.00157809,
        0.00172091, 0.00153971, 0.0014782 , 0.00165796, 0.00143218]),
 'score_time': array([0.00064898, 0.00032687, 0.00030279, 0.00029063, 0.00032783,
        0.00029206, 0.00029325, 0.00028276, 0.0003922 , 0.00029492]),
 'test_r2': array([0.78470494, 0.9094327 , 0.89492403, 0.88364959, 0.93120704,
        0.89646959, 0.92687378, 0.91605368, 0.89457282, 0.9389469 ]),
 'test_max_error': array([-9.88157631, -8.63180053, -8.74945258, -9.25512667, -8.90395527,
        -8.60605307, -8.58372959, -8.86187207, -8.45576112, -7.75666791])}

❓ Hesaplayın:
- Ortalama çapraz doğrulanmış R2 skoru ve bunu `r2` değişkeninde kaydedin
- Tüm katlarınızın °C cinsinden en büyük tek tahmin hatasını hesaplayın ve `max_error_celsius` değişkeninde kaydedin

(İpucu: `max_error` sklearn'de kabul edilen bir puanlama metriğidir)

In [6]:
r2 = sgd_model_cv['test_r2'].mean()
max_error_celsius = abs(sgd_model_cv['test_max_error']).max()
print(max_error_celsius)
print(r2)

9.881576309967063
0.8976835059199848


### Ortalama Mutlak Hata (MAE) Kaybı

Peki modelimizi MAE üzerinde optimize edersek ne olur?

❓ **MAE** Kaybı üzerinde **Stokastik Gradyan İnişi** (SGD) ile optimize edilmiş bir Doğrusal Regresyon modelini **10-Katlı Çapraz doğrula**

<details>
<summary>💡 İpuçları</summary>

- MAE kaybı `SGDRegressor`'da doğrudan belirtilemez. Doğru parametreleri ayarlayarak tasarlanması gerekir

</details>

In [7]:
mae_model = SGDRegressor(loss="epsilon_insensitive", epsilon = 0)
mae_sgd = cross_validate(mae_model, X_scaled,  data['Average Temperature'], cv = 10,  scoring = ['r2','max_error'])

❓ Hesaplayın:
- Ortalama çapraz doğrulanmış R2 skoru, bunu `r2_mae`'de saklayın
- Tüm katlarınızın en büyük tek tahmin hatasını, bunu `max_error_mae`'de saklayın

In [9]:
r2_mae = mae_sgd['test_r2'].mean()
max_error_mae = abs(mae_sgd['test_max_error']).max()
print(r2_mae)
print(max_error_mae)

0.8765487056787309
11.149032628517425


## 3. Sonuç

❓ Değerlendirdiğiniz modellerden hangisi göreviniz için en uygun görünüyor?

<details>
<summary> 🆘Cevap </summary>
    
İki model arasında ortalama çapraz doğrulanmış r2 skorları yaklaşık olarak benzer olmasına rağmen, MAE üzerinde optimize edilen modelin zaman zaman daha büyük hatalar yapma şansı daha fazladır, bu da bitkileri öldürme riskini artırır!
    
</details>

> MSE optimize edilen model daha uygun; çünkü hem R² daha yüksek hem de maksimum hata daha düşük. Bu görevde büyük hatalar kritik olduğu için MSE yaklaşımı daha güvenli.

# 🏁 Kodunuzu kontrol edin ve notebook'unuzu gönderin

In [10]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'loss_functions',
    r2 = r2,
    r2_mae = r2_mae,
    max_error = max_error_celsius,
    max_error_mae = max_error_mae
)

result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /Users/dilay/.pyenv/versions/workintech/bin/python
cachedir: .pytest_cache
rootdir: /Users/dilay/Desktop/S16D4-S-loss-functions/tests
plugins: dash-3.3.0, anyio-4.8.0, typeguard-4.4.2
collecting ... collected 3 items

test_loss_functions.py::TestLossFunctions::test_max_error_order PASSED   [ 33%]
test_loss_functions.py::TestLossFunctions::test_r2 PASSED                [ 66%]
test_loss_functions.py::TestLossFunctions::test_r2_mae PASSED            [100%]

============================== 3 passed in 0.11s ===============================


💯 You can commit your code:

git add tests/loss_functions.pickle

git commit -m 'Completed loss_functions step'

git push origin master

